In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
from xgboost import XGBRegressor
warnings.filterwarnings("ignore")


/usr/local/lib/python2.7/dist-packages/pandas/computation/expressions.py:21: UserWarning: The installed version of numexpr 1.4.2 is not supported in pandas and will be not be used
The minimum supported version is 2.1

  "version is 2.1\n".format(ver=ver), UserWarning)


In [2]:
def make_submission(answers, name):
    sample_submission = pd.read_csv("sample_submission.tsv")
    sample_submission['y'] = answers
    # In GBM you can get some negative predictions:
    print sample_submission[sample_submission['y'] < 0]
    sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)
    sample_submission.to_csv(name, sep=',', index=False)

In [3]:
def get_sMAPEError(y, y_pred):
    return 200 * np.sum(np.abs((y - y_pred) / (np.abs(y) + np.abs(y_pred)))) / y.size

In [4]:
hasher = {}
lb = preprocessing.LabelBinarizer()
back_lb = {}
back_hasher = {}
indexes = {}

In [5]:
def update_hash(column):
    global hasher
    for item in column:
        if item not in hasher:
            ttt = len(hasher)
            hasher[item] = ttt
            back_hasher[ttt] = item
def changed_column(column):
    global hasher
    new_column = np.array([hasher[item] for item in column])
    return new_column

In [6]:
def add_hash(column):
    global hasher
    for item in column:
        if item not in hasher:
            hasher[item] = item

In [7]:
def binarize_data(frame, i_p=3, o=True):
    frame_len = frame["item_id"].values.shape[0]

    binarized = lb.transform(frame["item_id"].values.astype(int))
    frame.drop('item_id', axis=1, inplace=True)
    insert_place = i_p

    for i in binarized.T:
        frame.insert(insert_place, "item_id == " + str(insert_place - i_p), i)
        insert_place += 1

In [8]:
def read_data(_frac=1.0):
    global hasher
    hasher = {}
    test = pd.read_csv("test.tsv")
    test = test[test["shift"] == 1]

    train = pd.read_csv("train.tsv")
    train = train.sample(frac=_frac, random_state=42)
    data = train[train["shift"] == 1]
    data.drop('Num', axis=1, inplace=True)
    test.drop('Num', axis=1, inplace=True)
    diff = np.setdiff1d(data["item_id"].values, test["item_id"].values)
    
    for i in diff:
        data.drop(data[data["item_id"] == i].index, inplace=True)
    
    update_hash(data['item_id'])
    update_hash(test['item_id'])
    data['item_id'] = changed_column(data['item_id'])
    test['item_id'] = changed_column(test['item_id'])
    
    data.sort(['item_id', 'year', 'week'], inplace=True)
    
    labels = data['y'] # * 0.62108408518104941
    data.drop('y', axis=1, inplace=True)
    
    
    lb.fit(np.unique(data["item_id"].values))
    
    for i in np.unique(data["item_id"].values):
        back_lb[''.join(lb.transform([i]).astype(str)[0])] = i
    binarize_data(data, o=False)
    binarize_data(test, 2)
    to_drop = ['f' + str(i) for i in np.arange(31, 61)]
    data.drop('shift', axis=1, inplace=True)
    test.drop('shift', axis=1, inplace=True)
    print(to_drop)
    for i in to_drop:
        data.drop(i, axis=1, inplace=True)
        test.drop(i, axis=1, inplace=True)

#     for i in np.arange(288, 350):
#         array = np.concatenate((data.iloc[:, [i]].values, test.iloc[:, [i]]), axis=0)
#         mean = np.mean(array)
#         sigma = np.sqrt(np.var(array))
#         data.iloc[:, [i]] = (data.iloc[:, [i]].values - mean) / sigma
#         test.iloc[:, [i]] = (test.iloc[:, [i]].values - mean) / sigma

    return data, labels, test

In [9]:
%%time
data, labels, test = read_data()

['f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60']
CPU times: user 6.21 s, sys: 1.13 s, total: 7.34 s
Wall time: 7.34 s


In [10]:
data.head()

,year,week,item_id == 0,item_id == 1,item_id == 2,item_id == 3,item_id == 4,item_id == 5,item_id == 6,item_id == 7,...,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30
76,2012,52,1,0,0,0,0,0,0,0,...,140980,127850,152370,114950,127800,126230,184760,98680,144500,131580
769,2013,1,1,0,0,0,0,0,0,0,...,127850,152370,114950,127800,126230,184760,98680,144500,131580,230260
1464,2013,2,1,0,0,0,0,0,0,0,...,152370,114950,127800,126230,184760,98680,144500,131580,230260,78000
2155,2013,3,1,0,0,0,0,0,0,0,...,114950,127800,126230,184760,98680,144500,131580,230260,78000,62150
2847,2013,4,1,0,0,0,0,0,0,0,...,127800,126230,184760,98680,144500,131580,230260,78000,62150,165635


In [11]:
test.head()

,year,week,item_id == 0,item_id == 1,item_id == 2,item_id == 3,item_id == 4,item_id == 5,item_id == 6,item_id == 7,...,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30
678,2015,3,0,0,0,0,0,0,0,0,...,969,1635,895,2140,1182,1020,1293,1290,422,1690
679,2015,3,0,0,0,0,0,0,0,0,...,21195,18280,18270,15851,16920,18320,24116,21307,6661,20610
680,2015,3,0,0,0,0,0,0,0,0,...,221622,256605,240047,236630,206697,245652,286179,285904,54846,171320
681,2015,3,0,0,0,0,0,0,0,0,...,22450,22093,31175,23355,15358,18930,29643,33970,7787,20395
682,2015,3,0,0,0,0,0,0,0,0,...,60,30,50,20,20,30,0,0,0,10


### Holdout

In [12]:
check_data = data[-1000:]
check_labels = labels[-1000:]
train_data = data[:-1000]
train_labels = labels[:-1000]
print (check_data.shape, check_labels.shape)
print (train_data.shape, train_labels.shape)

((1000, 259), (1000,))
((21275, 259), (21275,))


In [13]:
def cross_val_time_dependent(size, k=5):
    """
    [1] - [2]
    [1][2] - [3]
    [1][2][3] - [4]
    [1][2][3][4] - [5]
    """
    parts = [0] + [size / k] * (k - 1) + [size / k + size % k]
    parts = np.cumsum(parts)
    return np.array([(np.arange(parts[pc - 1]), np.arange(parts[pc - 1], parts[pc]))
                    for pc in np.arange(1, k + 1)])[1:]
    

In [14]:
def perform_cros_val(estimators, folds, fit_data=train_data, fit_labels=train_labels, o=False):
    f1=open("cros_val", 'w+')
    err = 0
    counter = 1
    err = np.zeros(folds - 1)
    for cv in cross_val_time_dependent(train_data.shape[0], folds):
        train_indices, test_indices = cv
        for estimator in estimators:
            estimator.fit(train_data.iloc[train_indices], train_labels.iloc[train_indices].values)
        mean_predicted = np.array(estimators[0].predict(train_data.iloc[test_indices]))
        
        for i in np.arange(1, len(estimators)):
            mean_predicted = np.vstack((mean_predicted, estimators[i].predict(train_data.iloc[test_indices])))
        if len(mean_predicted.shape) > 1:
            err[counter - 1] = get_sMAPEError(train_labels.iloc[test_indices].values, np.mean(mean_predicted, axis=0))
        else:
            err[counter - 1] = get_sMAPEError(train_labels.iloc[test_indices].values, mean_predicted)
        if o:
            print >>f1, "did %d folds" % counter
            
        counter += 1
    print >>f1, (np.sum(err) / (folds - 1), np.max(err), )
    f1.close()
    return (np.sum(err) / (folds - 1), np.max(err), )

In [15]:
def predict(estimator, t_d, splitted):
    if not splitted:
        return estimator.predict(t_d)
    ans = []
    for i in unique:
        ans += [estimator.predict(t_d)]
    ans = np.array(ans)
    return ans

In [16]:
def full_fit_predict_mean(estimators, fit_data=data, fit_labels=labels, target=test):
    for estimator in estimators:
            estimator.fit(fit_data, fit_labels.values)

    mean_predicted = np.array(estimators[0].predict(target))
    for i in np.arange(1, len(estimators)):
        mean_predicted = np.vstack((mean_predicted, estimators[i].predict(target)))
    if len(mean_predicted.shape) > 1:
        return np.mean(mean_predicted, axis=0)
    else:
        return mean_predicted

In [17]:
def check_real_error(estimators, fit_data, fit_labels, target):
    return get_sMAPEError(full_fit_predict_mean(estimators,
                                                fit_data,
                                                fit_labels, 
                                                target),
                         check_labels)

In [27]:
def perform_cros_val(estimators, folds, fit_data=train_data, fit_labels=train_labels, o=False,
                     filename="cross_val.txt"):
    err = 0
    counter = 1
    f1 = open(filename, "w")
    err = np.zeros(folds - 1)
    for cv in cross_val_time_dependent(train_data.shape[0], folds):
        train_indices, test_indices = cv
        for estimator in estimators:
            estimator.fit(train_data.iloc[train_indices], train_labels.iloc[train_indices].values)
        mean_predicted = np.array(estimators[0].predict(train_data.iloc[test_indices]))
        
        for i in np.arange(1, len(estimators)):
            mean_predicted = np.vstack((mean_predicted, estimators[i].predict(train_data.iloc[test_indices])))
        if len(mean_predicted.shape) > 1:
            err[counter - 1] = get_sMAPEError(train_labels.iloc[test_indices].values, np.mean(mean_predicted, axis=0))
        else:
            err[counter - 1] = get_sMAPEError(train_labels.iloc[test_indices].values, mean_predicted)
        if o:
            print >>f1, "did %d folds" % counter
            
        counter += 1
    print >>f1, (np.sum(err) / (folds - 1), np.max(err), )
    f1.close()  
    return (np.sum(err) / (folds - 1), np.max(err), )

In [19]:
def full_fit_predict_mean(estimators, fit_data=train_data, fit_labels=train_labels, target=test):
    for estimator in estimators:
            estimator.fit(fit_data, fit_labels.values)

    mean_predicted = np.array(estimators[0].predict(target))
    for i in np.arange(1, len(estimators)):
        mean_predicted = np.vstack((mean_predicted, estimators[i].predict(target)))
    if len(mean_predicted.shape) > 1:
        return np.mean(mean_predicted, axis=0)
    else:
        return mean_predicted

In [20]:
def check_real_error(estimators, fit_data, fit_labels, target):
    return get_sMAPEError(full_fit_predict_mean(estimators,
                                                fit_data,
                                                fit_labels, 
                                                target),
                         check_labels)

In [21]:
def append_estimator_answer(frame, estimator, name, fit_data, fit_labels, target):
    predicted = estimator.predict(target)
    new_data = frame
    new_data.insert(frame.shape[1], name + " answer", predicted)
    return new_data

In [43]:
xgb = XGBRegressor(n_estimators=5000, nthread=16, learning_rate=0.1, max_depth=10)
perform_cros_val([xgb], 6, o=True, filename="cross_val_xgb.txt")

KeyboardInterrupt: 

In [37]:
check_real_error([xgb], train_data, train_labels, target=check_data)

20.511363815313395

In [856]:
get_sMAPEError(full_fit_predict_mean([xgb], data, labels, data), labels)

30.098054224586303

In [58]:
from sklearn.ensemble import ExtraTreesRegressor
etreg = ExtraTreesRegressor(n_estimators=5000, n_jobs=16)

In [33]:
perform_cros_val([etreg], 6, o=True, filename="cv_etreg.txt")

(23.20944997826027, 26.634495569330543)

In [34]:
check_real_error([etreg, xgb], train_data, train_labels, target=check_data)

18.711812314630716

In [41]:
from sklearn.ensemble import BaggingRegressor
bag_xgboost = BaggingRegressor(base_estimator=XGBRegressor(n_estimators=5000,
                                                           max_depth=10, learning_rate=0.1), n_estimators=10)
perform_cros_val([bag_xgboost], 6, o=True,  filename="cross_val_xgb.txt")

(21.8050954642544, 23.978184366412915)

In [ ]:
predicted = full_fit_predict_mean([etreg], data, labels, test)

In [45]:
def renew_ans(df):
    d = {}
    for kk, i in enumerate(df.iterrows()):
        year = i[1]['year'].astype(int)
        week = (i[1]['week'] - i[1]['shift']).astype(int)
        item_id = hasher[i[1]['item_id']]
        f30 = (i[1]['f30']).astype(int)
        if week <= 0:
            year -= 1
            week += 52
        l = i[1][5:]
    
#         for k, j in enumerate(reversed(l)):
#             #print j

#             if week <= 0:
#                 year -= 1
#                 week += 52
#             d[(year, week, item_id)] = j.astype(int)
#             week -= 1
        d[(year, week, item_id)] = f30 / 0.6211
#    for i, j in d.iteritems():
#        print i, j
    return d

In [46]:
real_test = pd.read_csv("test.tsv")

In [47]:
d = renew_ans(real_test)

In [48]:
def build_pred(predicted, df):
#     nums_hash = { 
#         (real_test['year'][i], 
#          real_test['week'][i], 
#          hasher[real_test['item_id'][i]],
#         ):i
#          for i in np.arange(real_test.shape[0])}
# #    print(nums_hash)
#     for i, j in nums_hash.iteritems():
#         print i, j
#     real_answer = np.array([] * real_test.shape[0])
    
    ff = {}
    #print df.shape, predicted.shape

    for k, i in enumerate(df.iterrows()):
        year = i[1]['year'].astype(int)
        week = (i[1]['week']).astype(int)
        item_id = back_lb[''.join(i[1][2:229].astype(int).astype(str))]
        print year, week, item_id
        ff[(year, week, item_id)] = predicted[k]
        
#     for i in np.arange(df.shape[0]):
#         for j in [1, 2, 3]:
#             real_answer[nums_hash[df['year'][i], 
#                                   df['week'][i],
#                                   back_lb[''.join(df.iloc[i][2:288].values.astype(int).astype(str))],
#                                   j]] = predicted[i]
    return ff

In [49]:
ff = build_pred(predicted, test)

2015 3 208
2015 3 188
2015 3 137
2015 3 210
2015 3 122
2015 3 87
2015 3 12
2015 3 50
2015 3 121
2015 3 204
2015 3 81
2015 3 88
2015 3 58
2015 3 89
2015 3 153
2015 3 63
2015 3 78
2015 3 46
2015 3 25
2015 3 41
2015 3 147
2015 3 143
2015 3 107
2015 3 182
2015 3 45
2015 3 9
2015 3 31
2015 3 189
2015 3 151
2015 3 62
2015 3 192
2015 3 86
2015 3 18
2015 3 195
2015 3 183
2015 3 6
2015 3 28
2015 3 27
2015 3 199
2015 3 159
2015 3 1
2015 3 8
2015 3 119
2015 3 60
2015 3 133
2015 3 4
2015 3 148
2015 3 2
2015 3 90
2015 3 150
2015 3 167
2015 3 30
2015 3 96
2015 3 135
2015 3 36
2015 3 5
2015 3 29
2015 3 209
2015 3 190
2015 3 223
2015 3 126
2015 3 225
2015 3 94
2015 3 156
2015 3 222
2015 3 101
2015 3 224
2015 3 177
2015 3 120
2015 3 152
2015 3 185
2015 3 149
2015 3 0
2015 3 115
2015 3 186
2015 3 93
2015 3 197
2015 3 168
2015 3 111
2015 3 61
2015 3 117
2015 3 162
2015 3 205
2015 3 124
2015 3 216
2015 3 173
2015 3 123
2015 3 155
2015 3 82
2015 3 176
2015 3 79
2015 3 38
2015 3 174
2015 3 217
2015 3 180
20

In [50]:
ttt = pd.read_csv("test.tsv")

In [51]:
ttt['item_id'] = changed_column(ttt['item_id'])

In [52]:
test

,year,week,item_id == 0,item_id == 1,item_id == 2,item_id == 3,item_id == 4,item_id == 5,item_id == 6,item_id == 7,...,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30
678,2015,3,0,0,0,0,0,0,0,0,...,969,1635,895,2140,1182,1020,1293,1290,422,1690
679,2015,3,0,0,0,0,0,0,0,0,...,21195,18280,18270,15851,16920,18320,24116,21307,6661,20610
680,2015,3,0,0,0,0,0,0,0,0,...,221622,256605,240047,236630,206697,245652,286179,285904,54846,171320
681,2015,3,0,0,0,0,0,0,0,0,...,22450,22093,31175,23355,15358,18930,29643,33970,7787,20395
682,2015,3,0,0,0,0,0,0,0,0,...,60,30,50,20,20,30,0,0,0,10
683,2015,3,0,0,0,0,0,0,0,0,...,94634,91999,119165,96190,88666,118744,132368,117730,28229,87669
684,2015,3,0,0,0,0,0,0,0,0,...,63143,61417,63957,43027,53260,58029,62126,71857,17621,54582
685,2015,3,0,0,0,0,0,0,0,0,...,43432,52948,57199,60984,44887,58033,98839,76432,23853,46516
686,2015,3,0,0,0,0,0,0,0,0,...,45296,53534,58892,61250,54951,62347,106921,84194,23840,48798
687,2015,3,0,0,0,0,0,0,0,0,...,1610,2190,1464,1720,20460,1650,2568,1902,652,1618


In [53]:
def build_answer(ff):
    global real_test
    global d
    t = 0 
    n = 0
    new_y = np.arange(real_test.shape[0])
    for k, i in enumerate(real_test.iterrows()):
        year = (i[1]['year']).astype(int)
        week = (i[1]['week']).astype(int)
        item_id = hasher[i[1]['item_id']]
        
        if (year, week, item_id) in d:
            t += 1
            new_y[k] = d[(year, week, item_id)]
        else:
            n+=1
            new_y[k] = ff[(year, week, item_id)]
    print t, n
    return new_y

In [54]:
pred_no_shift = build_answer(ff)

1335 681


In [55]:
pred_no_shift

array([  1497,  27034, 297568, ...,  18775,    718,   6331])

In [56]:
make_submission(pred_no_shift, "xgb_5000.tsv")

         Num    y
455   349748 -563
625   349918 -142
654   349947 -984
907   350420 -563
1077  350590 -142
1106  350619 -984
1320  350833 -243
1353  350866 -563
1523  351036 -142
1552  351065 -984
1766  351279 -243
1986  351499 -243


In [41]:
test.head()

,year,week,item_id == 0,item_id == 1,item_id == 2,item_id == 3,item_id == 4,item_id == 5,item_id == 6,item_id == 7,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
0,2015,5,0,0,0,0,0,0,0,0,...,103505,96610,77869,102530,145088,143182,28000,76860,85980,110290
1,2015,4,0,0,0,0,0,0,0,0,...,98790,103505,96610,77869,102530,145088,143182,28000,76860,85980
2,2015,3,0,0,0,0,0,0,0,0,...,92830,98790,103505,96610,77869,102530,145088,143182,28000,76860
3,2015,5,0,0,0,0,0,0,0,0,...,3045,3019,3715,3324,4663,4238,620,2584,4385,3148
4,2015,4,0,0,0,0,0,0,0,0,...,2641,3045,3019,3715,3324,4663,4238,620,2584,4385


In [43]:
''.join(test.iloc[1][2:288].values.astype(int).astype(str))

'0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [40]:
lb.transform([10])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [149]:
test = pd.read_csv("test_1.tsv")

In [150]:
test.head()

,year,week,item_id,f,f_0,f1,f2,f3,f4,f5,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
0,2015,5,0,128552,114160,114000,113040,118870,134961,100060,...,103505,96610,77869,102530,145088,143182,28000,76860,85980,110290
1,2015,4,0,118340,128552,114160,114000,113040,118870,134961,...,98790,103505,96610,77869,102530,145088,143182,28000,76860,85980
2,2015,3,0,95170,118340,128552,114160,114000,113040,118870,...,92830,98790,103505,96610,77869,102530,145088,143182,28000,76860
3,2015,5,1,9950,6946,7705,7931,8128,10205,7261,...,3045,3019,3715,3324,4663,4238,620,2584,4385,3148
4,2015,4,1,7943,9950,6946,7705,7931,8128,10205,...,2641,3045,3019,3715,3324,4663,4238,620,2584,4385


In [102]:
np.unique(test['item_id'].values)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  39,  41,
        42,  43,  44,  45,  46,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  90,  91,  92,  93,  95,  96,  97,  98,
        99, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 117, 118, 119, 120, 121, 122, 123, 125, 126, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170,
       171, 172, 173, 174, 177, 178, 179, 180, 181, 182, 183, 184, 185,
       186, 187, 188, 190, 191, 192, 193, 194, 195, 196, 197, 19

In [103]:
np.unique(data['item_id'].values)

KeyError: 'item_id'

In [339]:
array = np.concatenate(([1, 2], [1, 2]), axis=0)

In [340]:
array

array([1, 2, 1, 2])

In [529]:
a = pd.read_csv("train.tsv")

In [525]:
a.shape

(72457, 66)

In [531]:
a = a.drop(a[a["item_id"]==20439191].index).shape

In [532]:
a[a["item_id"]==20439191].index

TypeError: tuple indices must be integers, not str